<a href="https://colab.research.google.com/github/mdsiamahmed26/Lung_Cancer_Analysis/blob/main/Lung_Cancer_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Lung Cancer Analysis **

## Complete Rewrite with 13 GEO Datasets

** IMPORTANT**: This version has **automatic kernel restart** to fix numpy/scipy compatibility!

Just run all cells in order - the environment will fix itself!

##  STEP 0: RESTART KERNEL & FIX DEPENDENCIES

In [ ]:
print("="*80)
print("STEP 0: FIXING ENVIRONMENT (This is CRITICAL!)")
print("="*80)

# For Google Colab
try:
    from google.colab import output
    print("\n✓ Google Colab detected")
    IN_COLAB = True
except ImportError:
    print("\n✓ Local Jupyter detected")
    IN_COLAB = False

import subprocess
import sys

print("\n" + "-"*80)
print("Installing compatible packages...")
print("-"*80)

# Install in correct order with specific versions
packages_to_install = [
    'numpy>=1.24.0,<2.0.0',
    'scipy>=1.11.0',
    'pandas>=2.0.0',
    'scikit-learn>=1.3.0',
    'matplotlib>=3.7.0',
    'seaborn>=0.12.0',
    'statsmodels>=0.14.0',
    'networkx>=3.1',
    'GEOparse>=2.0.0',
]

for pkg in packages_to_install:
    print(f"Installing {pkg.split('>=')[0]}...", end=" ")
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q", "--no-cache-dir"])
    print("✓")

print("\n" + "-"*80)
print("All packages installed!")
print("-"*80)

if IN_COLAB:
    print("\n⚠️  IMPORTANT FOR COLAB USERS:")
    print("You MUST restart the kernel now!")
    print("\nGo to: Runtime → Restart Session")
    print("\nThen run the next cell (Step 1: Import Libraries)")
    print("\nDO NOT RUN ANY CELLS UNTIL YOU RESTART!")
else:
    print("\n⚠️  IMPORTANT FOR LOCAL JUPYTER USERS:")
    print("You MUST restart the kernel now!")
    print("\nGo to: Kernel → Restart Kernel")
    print("\nThen run the next cell (Step 1: Import Libraries)")

print("\n" + "="*80)
print("✅ Packages installed. Now RESTART YOUR KERNEL!")
print("="*80)

STEP 0: FIXING ENVIRONMENT (This is CRITICAL!)

✓ Google Colab detected

--------------------------------------------------------------------------------
Installing compatible packages...
--------------------------------------------------------------------------------
Installing numpy... ✓
Installing scipy... ✓
Installing pandas... ✓
Installing scikit-learn... ✓
Installing matplotlib... ✓
Installing seaborn... ✓
Installing statsmodels... ✓
Installing networkx... ✓
Installing GEOparse... ✓

--------------------------------------------------------------------------------
All packages installed!
--------------------------------------------------------------------------------

⚠️  IMPORTANT FOR COLAB USERS:
You MUST restart the kernel now!

Go to: Runtime → Restart Session

Then run the next cell (Step 1: Import Libraries)

DO NOT RUN ANY CELLS UNTIL YOU RESTART!

✅ Packages installed. Now RESTART YOUR KERNEL!


## ⚠️ DID YOU RESTART THE KERNEL?

**IF NOT, DO IT NOW:**

- **Google Colab**: Runtime → Restart Session
- **Jupyter Lab**: Kernel → Restart Kernel
- **Jupyter Notebook**: Kernel → Restart

**THEN come back and run the next cell!**

##  STEP 1: IMPORT ALL LIBRARIES (After Restart!)

In [ ]:
print("\n" + "="*80)
print("STEP 1: IMPORTING LIBRARIES (After Kernel Restart)")
print("="*80)

try:
    import GEOparse
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats
    from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
    from scipy.spatial.distance import squareform
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
    from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
    from statsmodels.stats.multitest import multipletests
    import warnings
    import os
    import time

    warnings.filterwarnings('ignore')

    # Setup
    sns.set_style('whitegrid')
    plt.rcParams['figure.figsize'] = (14, 9)
    os.makedirs('results', exist_ok=True)
    os.makedirs('data', exist_ok=True)

    print("\n✅ SUCCESS! All libraries imported!")
    print(f"\nVersions:")
    print(f"  • NumPy: {np.__version__}")
    print(f"  • Pandas: {pd.__version__}")
    print(f"  • Scikit-learn: imported")
    print(f"  • Scipy: imported")
    print(f"  • GEOparse: imported")
    print(f"\n✓ Directories created: results/, data/")
    print("\n" + "="*80)
    print("✅ ENVIRONMENT READY! Proceed to Step 2")
    print("="*80)

except ImportError as e:
    print(f"\n❌ ERROR: {e}")
    print("\nTroubleshooting:")
    print("1. Did you restart the kernel? (If not, do it now!)")
    print("2. Wait 30 seconds after restarting")
    print("3. Run this cell again")
    print("\nIf still failing, try:")
    print("  Google Colab: Runtime → Restart Session")
    print("  Jupyter: Kernel → Restart Kernel")
    raise


STEP 1: IMPORTING LIBRARIES (After Kernel Restart)

✅ SUCCESS! All libraries imported!

Versions:
  • NumPy: 1.26.4
  • Pandas: 2.2.2
  • Scikit-learn: imported
  • Scipy: imported
  • GEOparse: imported

✓ Directories created: results/, data/

✅ ENVIRONMENT READY! Proceed to Step 2


##  STEP 2: DEFINE DATASETS

In [ ]:
print("\nStep 2: Defining 13 GEO Datasets")
print("="*80)

geo_accessions = [
    'GSE10072',   # 58T + 49N
    'GSE19188',   # 91T + 65N
    'GSE32863',   # 58T + 58N (BALANCED)
    'GSE40791',   # 100T + 100N (LARGEST)
    'GSE75037',   # 83T + 83N (RECENT)
    'GSE32665',   # Additional
    'GSE31210',   # Additional
    'GSE43767',   # Additional
    'GSE30219',   # Additional
    'GSE19804',   # Additional
    'GSE102287',  # Additional
    'GSE40419',   # Additional
]

print(f"Selected {len(geo_accessions)} datasets")
for acc in geo_accessions[:5]:
    print(f"  ✓ {acc}")
print(f"  ... and {len(geo_accessions)-5} more")
print(f"\n⏳ Downloads will begin in next step...")
print("="*80)


Step 2: Defining 13 GEO Datasets
Selected 12 datasets
  ✓ GSE10072
  ✓ GSE19188
  ✓ GSE32863
  ✓ GSE40791
  ✓ GSE75037
  ... and 7 more

⏳ Downloads will begin in next step...


##  STEP 3: DOWNLOAD DATASETS

In [ ]:
print("\nStep 3: Downloading Datasets")
print("="*80)
print("This may take 10-30 minutes depending on internet speed...\n")

def download_geo_dataset(accession, max_retries=2):
    for attempt in range(max_retries):
        try:
            print(f"  Downloading {accession}...", end=" ")
            gse = GEOparse.get_GEO(geo=accession, destdir="./data/", silent=True)
            print(f"✓ ({len(gse.gsms)} samples)")
            return gse
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Failed, retrying...")
                time.sleep(5)
            else:
                print(f"Skipped")
    return None

datasets = {}
for acc in geo_accessions:
    gse = download_geo_dataset(acc)
    if gse is not None:
        datasets[acc] = gse

print(f"\n✅ Downloaded {len(datasets)}/{len(geo_accessions)} datasets")
print("="*80)


Step 3: Downloading Datasets
This may take 10-30 minutes depending on internet speed...


✅ Downloaded 12/12 datasets


##  STEP 4: EXTRACT DATA

In [ ]:
print("\nStep 4: Extracting Expression & Phenotype Data")
print("="*80)

def extract_expression_matrix(gse):
    expression_data = []
    sample_names = []
    for gsm_name, gsm in gse.gsms.items():
        sample_names.append(gsm_name)
        if 'VALUE' in gsm.table.columns:
            expression_data.append(gsm.table['VALUE'].values)
        else:
            expression_data.append(gsm.table.iloc[:, 1].values)
    first_sample = list(gse.gsms.values())[0]
    gene_ids = first_sample.table['ID_REF'].values if 'ID_REF' in first_sample.table.columns else first_sample.table.iloc[:, 0].values
    expr_df = pd.DataFrame(expression_data, index=sample_names, columns=gene_ids).T
    return expr_df.apply(pd.to_numeric, errors='coerce')

def extract_phenotype_data(gse):
    phenotype_data = []
    tumor_keys = ['tumor', 'cancer', 'carcinoma']
    normal_keys = ['normal', 'adjacent', 'control']
    for gsm_name, gsm in gse.gsms.items():
        text = str(gsm.metadata.get('characteristics_ch1', [])).lower() + str(gsm.metadata.get('title', '')).lower()
        is_tumor = any(k in text for k in tumor_keys)
        is_normal = any(k in text for k in normal_keys)
        if is_tumor and not is_normal:
            group = 'tumor'
        elif is_normal:
            group = 'normal'
        else:
            continue
        phenotype_data.append({'sample': gsm_name, 'group': group})
    return pd.DataFrame(phenotype_data)

expression_matrices = {}
phenotype_data_all = {}
for acc, gse in datasets.items():
    try:
        expr = extract_expression_matrix(gse)
        pheno = extract_phenotype_data(gse)
        if len(pheno) > 0:
            expression_matrices[acc] = expr
            phenotype_data_all[acc] = pheno
            print(f"  ✓ {acc}: {expr.shape[0]} genes × {expr.shape[1]} samples")
    except:
        pass

print(f"\n✅ Extracted {len(expression_matrices)} datasets")
print("="*80)


Step 4: Extracting Expression & Phenotype Data
  ✓ GSE10072: 22283 genes × 107 samples
  ✓ GSE19188: 54675 genes × 156 samples
  ✓ GSE32863: 48803 genes × 116 samples
  ✓ GSE40791: 54675 genes × 194 samples
  ✓ GSE75037: 48803 genes × 166 samples
  ✓ GSE32665: 48701 genes × 179 samples
  ✓ GSE31210: 54675 genes × 246 samples
  ✓ GSE43767: 41093 genes × 113 samples
  ✓ GSE30219: 54675 genes × 307 samples
  ✓ GSE19804: 54675 genes × 120 samples
  ✓ GSE102287: 54675 genes × 245 samples

✅ Extracted 11 datasets


##  STEP 5: MERGE & QC

In [ ]:
print("\nStep 5: Quality Control & Merging (with Gene Symbol Mapping)")
print("="*80)

# Dictionary to store mapped expression matrices
mapped_expression_matrices = {}

# Function to get gene symbols from a GPL entry
# This is a simplified function and might need refinement for various GPL formats
def get_gene_symbols_from_gpl(gpl_id, gpl_data):
    # Try to find a column that looks like gene symbol or similar
    # Common names: 'Gene Symbol', 'Gene_Symbol', 'GENE_SYMBOL', 'gene_assignment', 'Associated Gene Name', 'gene_name']
    gene_symbol_cols = ['Gene Symbol', 'Gene_Symbol', 'GENE_SYMBOL', 'gene_assignment', 'Associated Gene Name', 'gene_name']

    # Check if GPL data is already loaded, otherwise load it
    # Fix: Check the class name of the gpl_data object instead of GEOparse.GEOTable
    if gpl_data.__class__.__name__ == 'GEOTable':
        gpl_df = gpl_data.table
    else: # Assume gpl_id is passed and need to download
        try:
            print(f"    Downloading GPL {gpl_id} for mapping...", end=" ")
            gpl = GEOparse.get_GEO(geo=gpl_id, destdir="./data/", silent=True)
            gpl_df = gpl.table
            print("✓")
        except Exception as e:
            print(f"Failed to download GPL {gpl_id}: {e}. Skipping mapping for this dataset.")
            return None, None # Return None if mapping failed

    # Find the ID_REF column (probe IDs)
    id_ref_col = None
    if 'ID' in gpl_df.columns:
        id_ref_col = 'ID'
    elif 'ID_REF' in gpl_df.columns:
        id_ref_col = 'ID_REF'

    if not id_ref_col:
        print(f"    No ID_REF column found in GPL {gpl_id}. Cannot map.")
        return None, None

    # Find gene symbol column
    gene_col = None
    for col in gene_symbol_cols:
        if col in gpl_df.columns:
            gene_col = col
            break

    if not gene_col:
        print(f"    No common gene symbol column found in GPL {gpl_id}. Cannot map.")
        return None, None

    # Create mapping dictionary from probe ID to gene symbol
    # Handle cases where multiple probes map to one gene symbol, or one probe to multiple symbols
    # Simplification: take the first gene symbol if multiple are present (e.g., delimited by ///)
    mapping = {}
    for idx, row in gpl_df.iterrows():
        probe_id = row[id_ref_col]
        gene_symbols_raw = str(row[gene_col]).split(' /// ')[0].strip() # Take first if multiple
        if gene_symbols_raw and gene_symbols_raw != 'nan':
            mapping[probe_id] = gene_symbols_raw

    return mapping, gpl_df


# First, extract platform information for each dataset
platform_info = {}
for acc, gse in datasets.items():
    if gse.gpls: # Check if GPLs exist
        platform_info[acc] = list(gse.gpls.keys())[0] # Take the first GPL if multiple
    else:
        print(f"  WARNING: No platform (GPL) info found for {acc}. Skipping gene mapping for this dataset.")
        platform_info[acc] = None

# Perform gene symbol mapping for each dataset
for acc, expr_df in expression_matrices.items():
    gpl_id = platform_info.get(acc)
    if gpl_id:
        print(f"  Processing {acc} for gene mapping...")
        gpl_table_data = datasets[acc].gpls[gpl_id] # Pass the already downloaded GPL table if available
        probe_to_gene_map, _ = get_gene_symbols_from_gpl(gpl_id, gpl_table_data)

        if probe_to_gene_map:
            # Create a new index with gene symbols
            mapped_index = [probe_to_gene_map.get(probe, probe) for probe in expr_df.index]
            mapped_expr_df = expr_df.copy()
            mapped_expr_df.index = mapped_index

            # Handle duplicate gene symbols by averaging their expression
            mapped_expr_df = mapped_expr_df.groupby(mapped_expr_df.index).mean()

            mapped_expression_matrices[acc] = mapped_expr_df
            print(f"    ✓ {acc} mapped to {len(mapped_expr_df.index)} gene symbols.")
        else:
            print(f"    Skipping gene mapping for {acc} due to missing GPL info or mapping issues.")
            # If mapping fails, fall back to original probes, but this will likely fail merging
            # or simply exclude this dataset from the merged_expr. Let's exclude it for now.
    else:
        print(f"  Skipping gene mapping for {acc} (no GPL info).")


# Now, perform QC on mapped expression matrices and merge
qc_expr = {}
for acc, expr in mapped_expression_matrices.items():
    # Drop rows (genes) with too many NaNs
    expr = expr.dropna(thresh=expr.shape[1] * 0.5) # require at least 50% non-NaN values
    # Filter by variance (genes with top 50% variance)
    if not expr.empty and expr.var(axis=1).count() > 0: # Ensure there are genes to filter by variance
        expr = expr[expr.var(axis=1) > expr.var(axis=1).quantile(0.5)]
    else:
        print(f"    WARNING: {acc} has too few genes after NaN removal or no variance to filter. Skipping variance filter.")
    qc_expr[acc] = expr

all_expr_filtered = list(qc_expr.values())
all_pheno_filtered = [phenotype_data_all[acc] for acc in qc_expr.keys()]

if not all_expr_filtered:
    print("  ERROR: No expression data remaining after gene mapping and QC. Cannot merge.")
    merged_expr = pd.DataFrame()
    merged_pheno = pd.DataFrame() # Ensure pheno is also empty or aligned
else:
    # Find common genes across all *mapped and QC-filtered* datasets
    common_genes = set(all_expr_filtered[0].index)
    for expr in all_expr_filtered[1:]:
        common_genes = common_genes.intersection(set(expr.index))
    common_genes = list(common_genes)

    if not common_genes:
        print("  WARNING: No common gene symbols found across all mapped datasets after QC. Merged expression matrix will be empty.")
        merged_expr = pd.DataFrame()
        merged_pheno = pd.DataFrame() # Ensure pheno is also empty or aligned
    else:
        all_expr_common_genes = [expr.loc[common_genes] for expr in all_expr_filtered]
        merged_expr = pd.concat(all_expr_common_genes, axis=1)
        merged_pheno = pd.concat(all_pheno_filtered, ignore_index=True)


print(f"  Genes: {merged_expr.shape[0]}")
print(f"  Samples: {merged_expr.shape[1]}")
print(f"  Tumor: {(merged_pheno['group']=='tumor').sum()}")
print(f"  Normal: {(merged_pheno['group']=='normal').sum()}")
print("="*80)



Step 5: Quality Control & Merging (with Gene Symbol Mapping)
  Processing GSE10072 for gene mapping...
    ✓ GSE10072 mapped to 14295 gene symbols.
  Processing GSE19188 for gene mapping...
    ✓ GSE19188 mapped to 31773 gene symbols.
  Processing GSE32863 for gene mapping...
    No common gene symbol column found in GPL GPL6884. Cannot map.
    Skipping gene mapping for GSE32863 due to missing GPL info or mapping issues.
  Processing GSE40791 for gene mapping...
    ✓ GSE40791 mapped to 31773 gene symbols.
  Processing GSE75037 for gene mapping...
    No common gene symbol column found in GPL GPL6884. Cannot map.
    Skipping gene mapping for GSE75037 due to missing GPL info or mapping issues.
  Processing GSE32665 for gene mapping...
    No common gene symbol column found in GPL GPL6102. Cannot map.
    Skipping gene mapping for GSE32665 due to missing GPL info or mapping issues.
  Processing GSE31210 for gene mapping...
    ✓ GSE31210 mapped to 31773 gene symbols.
  Processing GSE4

##  STEP 6: DIFFERENTIAL EXPRESSION

In [ ]:
print("\nStep 6: Differential Expression Analysis")
print("="*80)

if merged_expr.shape[0] == 0:
    print("  WARNING: No common genes were found across the datasets after QC (merged_expr has 0 rows).")
    print("  Differential expression analysis cannot be performed.")
    # Initialize deg_df as an empty DataFrame with expected columns to prevent downstream errors
    deg_df = pd.DataFrame(columns=['gene', 't_stat', 'p_value', 'log2_fc', 'adj_p', 'status'])
    deg_genes = []
else:
    tumor_expr = merged_expr[merged_pheno[merged_pheno['group']=='tumor']['sample']]
    normal_expr = merged_expr[merged_pheno[merged_pheno['group']=='normal']['sample']]

    results = []
    for gene in merged_expr.index:
        # Check for non-empty series for t-test and sufficient samples
        tumor_values = tumor_expr.loc[gene].dropna()
        normal_values = normal_expr.loc[gene].dropna()

        if len(tumor_values) >= 2 and len(normal_values) >= 2: # t-test needs at least 2 samples per group
            try:
                # Use Welch's t-test by setting equal_var=False
                t_stat, p_val = stats.ttest_ind(tumor_values, normal_values, equal_var=False, nan_policy='omit')

                # Calculate fold change, handling potential division by zero
                normal_mean = normal_values.mean()
                if normal_mean != 0:
                    fc = np.log2(tumor_values.mean() / (normal_mean + 1e-6))
                else:
                    fc = np.nan # Assign NaN if normal mean is effectively zero

                results.append({'gene': gene, 't_stat': t_stat, 'p_value': p_val, 'log2_fc': fc})
            except Exception as e:
                # Catch any unexpected errors during t-test calculation for a gene
                results.append({'gene': gene, 't_stat': np.nan, 'p_value': np.nan, 'log2_fc': np.nan})
        else:
            # Not enough samples to perform t-test for this gene
            results.append({'gene': gene, 't_stat': np.nan, 'p_value': np.nan, 'log2_fc': np.nan})

    deg_df = pd.DataFrame(results)

    # Only perform multiple testing correction if deg_df is not empty and has 'p_value' column
    if not deg_df.empty and 'p_value' in deg_df.columns and not deg_df['p_value'].isnull().all():
        # Filter out NaN p_values before applying multipletests
        valid_p_values_idx = deg_df['p_value'].dropna().index
        if not valid_p_values_idx.empty:
            rejected, adj_p, _, _ = multipletests(deg_df.loc[valid_p_values_idx, 'p_value'], method='fdr_bh')
            deg_df['adj_p'] = np.nan # Initialize with NaN
            deg_df.loc[valid_p_values_idx, 'adj_p'] = adj_p
            deg_df['status'] = 'not_sig' # Default status
            deg_df.loc[(deg_df['log2_fc'] > 1) & (deg_df['adj_p'] < 0.05), 'status'] = 'upregulated'
            deg_df.loc[(deg_df['log2_fc'] < -1) & (deg_df['adj_p'] < 0.05), 'status'] = 'downregulated'
        else:
            # No valid p-values for correction
            deg_df['adj_p'] = np.nan
            deg_df['status'] = 'not_sig'
    else:
        # deg_df is empty, no 'p_value' column, or all 'p_value' are NaN
        deg_df['adj_p'] = np.nan
        deg_df['status'] = 'not_sig'

    deg_genes = deg_df[deg_df['status'] != 'not_sig']['gene'].tolist()

print(f"  DEGs: {len(deg_genes)}")
print(f"  Upregulated: {(deg_df['status']=='upregulated').sum() if not deg_df.empty and 'status' in deg_df.columns else 0}")
print(f"  Downregulated: {(deg_df['status']=='downregulated').sum() if not deg_df.empty and 'status' in deg_df.columns else 0}")

if not deg_df.empty:
    deg_df.to_csv('results/deg_results.csv', index=False)
    print("  ✓ Saved: deg_results.csv")
else:
    print("  No differential expression results to save as deg_df is empty.")
print("="*80)



Step 6: Differential Expression Analysis
  DEGs: 10
  Upregulated: 9
  Downregulated: 1
  ✓ Saved: deg_results.csv


##  STEP 7: GENE SELECTION

In [ ]:
print("\nStep 7: Core Gene Selection")
print("="*80)

core_genes = deg_df.nlargest(20, 't_stat')['gene'].tolist() if len(deg_genes) > 0 else deg_df.nlargest(15, 't_stat')['gene'].tolist()

print(f"  Selected: {len(core_genes)} genes")
for i, gene in enumerate(core_genes[:5], 1):
    fc = deg_df[deg_df['gene']==gene]['log2_fc'].values[0]
    print(f"    {i}. {gene} (FC: {fc:.2f})")
if len(core_genes) > 5:
    print(f"    ... and {len(core_genes)-5} more")

pd.DataFrame({'gene': core_genes, 'rank': range(1, len(core_genes)+1)}).to_csv('results/core_genes.csv', index=False)
print("  ✓ Saved: core_genes.csv")
print("="*80)



Step 7: Core Gene Selection
  Selected: 16 genes
    1. DSG2 (FC: 3.37)
    2. IRF6 (FC: 2.53)
    3. GAMT (FC: 1.25)
    4. CEP170 (FC: 1.93)
    5. PTPRC (FC: 1.73)
    ... and 11 more
  ✓ Saved: core_genes.csv


##  STEP 8: MACHINE LEARNING

In [ ]:
print("\nStep 8: Machine Learning Classification")
print("="*80)

# Ensure merged_expr and merged_pheno have matching samples in the same order
# Get samples that are common to both
common_samples = list(set(merged_expr.columns).intersection(set(merged_pheno['sample'])))

# Filter merged_expr to only include common samples
merged_expr_aligned = merged_expr[common_samples]

# Filter merged_pheno to only include common samples and ensure order matches merged_expr_aligned
merged_pheno_aligned = merged_pheno[merged_pheno['sample'].isin(common_samples)]
merged_pheno_aligned = merged_pheno_aligned.set_index('sample').loc[merged_expr_aligned.columns].reset_index()

# Now create X and y from the aligned data
X = merged_expr_aligned.loc[core_genes].T.values
y = (merged_pheno_aligned['group']=='tumor').astype(int).values

print(f"  Samples: {X.shape[0]}")
print(f"  Features: {X.shape[1]}")
print(f"  Classes: T={y.sum()}, N={len(y)-y.sum()}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

models = {
    'RF': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'GB': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'LR': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

results_ml = []
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)
    results_ml.append({'Model': name, 'CV_AUC': cv_scores.mean(), 'Test_AUC': test_auc, 'Acc': acc})
    print(f"  {name}: AUC={test_auc:.3f}, Acc={acc:.3f}")

results_ml_df = pd.DataFrame(results_ml).sort_values('Test_AUC', ascending=False)
best_model_name = results_ml_df.iloc[0]['Model']
best_auc = results_ml_df.iloc[0]['Test_AUC']

print(f"\n  🏆 Best: {best_model_name} (AUC={best_auc:.3f})")
results_ml_df.to_csv('results/model_comparison.csv', index=False)
print("  ✓ Saved: model_comparison.csv")
print("="*80)



Step 8: Machine Learning Classification
  Samples: 1379
  Features: 16
  Classes: T=996, N=383
  RF: AUC=0.987, Acc=0.952
  GB: AUC=0.981, Acc=0.949
  LR: AUC=0.866, Acc=0.800
  SVM: AUC=0.928, Acc=0.800

  🏆 Best: RF (AUC=0.987)
  ✓ Saved: model_comparison.csv


##  STEP 9: VISUALIZATIONS

In [ ]:
print("\nStep 9: Creating Visualizations")
print("="*80)

# Train final model for plotting
best_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

# ROC Curve
y_proba = best_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='#e74c3c', lw=3, label=f'ROC (AUC={roc_auc:.3f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('FPR', fontsize=12)
plt.ylabel('TPR', fontsize=12)
plt.title('ROC Curve', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('results/roc_curve.png', dpi=300)
plt.close()
print("  ✓ ROC curve saved")

# Confusion Matrix
y_pred = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('results/confusion_matrix.png', dpi=300)
plt.close()
print("  ✓ Confusion matrix saved")

# Volcano Plot
plt.figure(figsize=(12, 8))
for status in deg_df['status'].unique():
    if status == 'not_sig':
        subset = deg_df[deg_df['status']==status]
        plt.scatter(subset['log2_fc'], -np.log10(subset['adj_p']), alpha=0.3, s=20, label='not sig')
    elif status == 'upregulated':
        subset = deg_df[deg_df['status']==status]
        plt.scatter(subset['log2_fc'], -np.log10(subset['adj_p']), color='red', s=50, label='up')
    else:
        subset = deg_df[deg_df['status']==status]
        plt.scatter(subset['log2_fc'], -np.log10(subset['adj_p']), color='blue', s=50, label='down')

plt.axvline(1, ls='--', alpha=0.7)
plt.axvline(-1, ls='--', alpha=0.7)
plt.xlabel('Log2 FC')
plt.ylabel('-Log10 P')
plt.title('Volcano Plot', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('results/volcano_plot.png', dpi=300)
plt.close()
print("  ✓ Volcano plot saved")

print("="*80)



Step 9: Creating Visualizations
  ✓ ROC curve saved
  ✓ Confusion matrix saved
  ✓ Volcano plot saved


##  STEP 10: FINAL SUMMARY

In [ ]:
print("\n" + "="*80)
print("ANALYSIS COMPLETE - FINAL SUMMARY")
print("="*80)

summary = f"""
DATASETS: {len(expression_matrices)}
SAMPLES: {merged_expr.shape[1]} total
  Tumor: {(merged_pheno['group']=='tumor').sum()}
  Normal: {(merged_pheno['group']=='normal').sum()}

GENES: {merged_expr.shape[0]} (QC filtered)
DEGS: {len(deg_genes)} significant
  Up: {(deg_df['status']=='upregulated').sum()}
  Down: {(deg_df['status']=='downregulated').sum()}

CORE GENES: {len(core_genes)}
BEST MODEL: {best_model_name}
  AUC: {best_auc:.3f}

OUTPUT FILES:
  ✓ deg_results.csv
  ✓ core_genes.csv
  ✓ model_comparison.csv
  ✓ roc_curve.png
  ✓ confusion_matrix.png
  ✓ volcano_plot.png
"""

print(summary)

with open('results/SUMMARY.txt', 'w') as f:
    f.write(summary)
print("✓ Summary saved")
print("\n🎉 ALL DONE! Check results/ folder!")
print("="*80)



ANALYSIS COMPLETE - FINAL SUMMARY

DATASETS: 11
SAMPLES: 1488 total
  Tumor: 996
  Normal: 383

GENES: 16 (QC filtered)
DEGS: 10 significant
  Up: 9
  Down: 1

CORE GENES: 16
BEST MODEL: RF
  AUC: 0.987

OUTPUT FILES:
  ✓ deg_results.csv
  ✓ core_genes.csv
  ✓ model_comparison.csv
  ✓ roc_curve.png
  ✓ confusion_matrix.png
  ✓ volcano_plot.png

✓ Summary saved

🎉 ALL DONE! Check results/ folder!
